# Imports

In [ ]:
!python ../src/cleaning.py

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

import sys
from pathlib import Path

c:\Users\rcarlisle\OneDrive - qcusd.org\Reports\Summer Bridge\2026 Summer Bridge\src\cleaning.py:75: DtypeWarning: Columns (0: Unit4UUID, 1: Oral Reading UIN, 2: Form Code - Unit 4, 3: U4 Attempt, 4: Move On When Reading Requirement, 5: RC1 C4 Cluster Code, 6: RC1 C5 Cluster Code, 7: RC2 C4 Cluster Code, 8: RC3 C4 Cluster Code, 9: RC3 C5 Cluster Code, 10: ORF UIN 1, 11: ORF Status 1, 12: ORF Score 1, 13: ORF UIN 3, 14: ORF Status 3, 15: ORF Score 3, 16: Read Aloud Test Content, 17: Simplified Directions, 18: IEP Designated Magnification, 19: Proctor Name, 20: Proctor Certified, 21: Proctor Selected Magnification) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, delimiter="\t", dtype={"SSID": str})


# Paths

In [26]:
src_path = Path.cwd().parent / "src"
sys.path.insert(0, str(src_path))
from config import DATA_PROCESSED

FILE = DATA_PROCESSED / "analysis_file.csv"

# Load Data

In [27]:
df = pd.read_csv(FILE)

print(df.shape)
df.head()

(445, 23)


,student_id,school,intervention,school_name,state_student_id,Gender,Grade,RaceEthnicity,SPED,FRL,...,Gifted,Migrant,CTE,DatePulled,subject_x,pretest_score,subject_y,bm1_score,ly_math_AASA_score,BM1_gain_score
0,440605,CHS,In-Person,Crismon High School,18646683,Male,9,White,N,N,...,N,N,N,2026-01-27T08:01:52-07:00,Algebra_I,3676.0,Algebra_I,3669.0,3653.0,-7.0
1,434131,CHS,In-Person,Crismon High School,62425336,Female,9,White,N,N,...,N,N,N,2026-01-27T08:01:52-07:00,Algebra_I,3674.0,Algebra_I,3678.0,3653.0,4.0
2,435873,CHS,In-Person,Crismon High School,49872954,Female,9,White,N,N,...,N,N,N,2026-01-27T08:01:52-07:00,Algebra_I,3662.0,Algebra_I,3670.0,3644.0,8.0
3,419323,CHS,In-Person,Crismon High School,12196385,Female,9,White,N,N,...,N,N,N,2026-01-27T08:01:52-07:00,Algebra_I,3657.0,Algebra_I,3651.0,3630.0,-6.0
4,421999,CHS,In-Person,Crismon High School,39448678,Male,9,White,N,N,...,N,N,N,2026-01-27T08:01:52-07:00,Algebra_I,3660.0,Algebra_I,3686.0,3671.0,26.0


# Quick Sanity Checks

In [5]:
df.info()
df.describe()
df.isna().mean().sort_values(ascending=False).head(10)

<class 'pandas.DataFrame'>
RangeIndex: 531 entries, 0 to 530
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   student_id          531 non-null    int64  
 1   school              531 non-null    str    
 2   intervention        531 non-null    str    
 3   school_name         445 non-null    str    
 4   state_student_id    445 non-null    float64
 5   Gender              445 non-null    str    
 6   Grade               445 non-null    float64
 7   RaceEthnicity       445 non-null    str    
 8   SPED                445 non-null    str    
 9   FRL                 445 non-null    str    
 10  504                 445 non-null    str    
 11  ELL                 445 non-null    str    
 12  EVIT                445 non-null    str    
 13  Gifted              445 non-null    str    
 14  Migrant             445 non-null    str    
 15  CTE                 445 non-null    str    
 16  DatePulled         

BM1_gain_score        0.175141
ly_math_AASA_score    0.175141
Gender                0.161959
Grade                 0.161959
RaceEthnicity         0.161959
state_student_id      0.161959
school_name           0.161959
Migrant               0.161959
Gifted                0.161959
CTE                   0.161959
dtype: float64

In [6]:
# Baseline Equivalence Analysis

In [36]:
baseline_vars = ["pretest_score", "ly_math_AASA_score"]

desc = (
    df.groupby("intervention")[baseline_vars]
      .agg(["mean", "std", "count"])
)

print(desc)

             pretest_score                  ly_math_AASA_score             \
                      mean        std count               mean        std   
intervention                                                                
In-Person      3662.300000  16.532226    70        3638.485714  16.191555   
Lab            3662.105263  18.315950   304        3638.853896  19.130822   
Online         3669.050000  22.850602    60        3651.800000  20.038776   

                    
             count  
intervention        
In-Person       70  
Lab            308  
Online          60  


# One-way ANOVA

In [37]:
import statsmodels.api as sm

for var in baseline_vars:
    model = smf.ols(f"{var} ~ C(intervention)", data=df).fit()
    print(var)
    print(sm.stats.anova_lm(model, typ=2))

pretest_score
                        sum_sq     df         F    PR(>F)
C(intervention)    2469.753905    2.0  3.517396  0.030531
Residual         151314.181579  431.0       NaN       NaN
ly_math_AASA_score
                        sum_sq     df          F    PR(>F)
C(intervention)    8777.934167    2.0  12.386186  0.000006
Residual         154139.511039  435.0        NaN       NaN


# Standardized Mean differences
Baseline comparisons across intervention groups indicated that the Lab and Online groups differed substantially in prior achievement (AASA SMD = 0.67–0.69). Pretest scores showed moderate differences (SMD ≈ 0.35) between Lab and Online. In-Person students were similar to Lab students at baseline. All outcome analyses controlled for pretest and prior-year achievement to account for these imbalances.

In [9]:
def smd(x, g, g1, g0):
    m1 = x[g==g1].mean()
    m0 = x[g==g0].mean()
    sd = x.std()
    return (m1 - m0) / sd

groups = df["intervention"].unique()

for i in groups:
    for j in groups:
        if i < j:
            print(i, "vs", j)
            print("  pretest:", smd(df["pretest_score"], df["intervention"], i, j))
            print("  AASA:", smd(df["ly_math_AASA_score"], df["intervention"], i, j))

In-Person vs Online
  pretest: -0.3471929938773957
  AASA: -0.6895643414564385
In-Person vs Lab
  pretest: 0.004996733393502238
  AASA: -0.019068619859816976
Lab vs Online
  pretest: -0.352189727270898
  AASA: -0.6704957215966216


# Primary Impact Models
Regression analyses controlling for pretest scores and prior-year achievement indicated no significant differences in BM1 scores or growth between students in the Lab, In-Person, or Online interventions. This suggests that all three interventions were similarly effective (or ineffective) in supporting algebra readiness for students in this cohort.

Growth score differences

In [40]:
df['intervention'] = df['intervention'].astype('category')

df["intervention"] = df["intervention"].cat.reorder_categories(
    ["Lab", "In-Person", "Online"],
    ordered=False
)

model_growth = smf.ols(
    "BM1_gain_score ~ C(intervention) + pretest_score + ly_math_AASA_score",
    data=df
).fit(cov_type="HC3")

print(model_growth.summary())

                            OLS Regression Results                            
Dep. Variable:         BM1_gain_score   R-squared:                       0.424
Model:                            OLS   Adj. R-squared:                  0.418
Method:                 Least Squares   F-statistic:                     93.47
Date:                Tue, 03 Feb 2026   Prob (F-statistic):           1.10e-56
Time:                        15:44:16   Log-Likelihood:                -1815.3
No. Observations:                 422   AIC:                             3641.
Df Residuals:                     417   BIC:                             3661.
Df Model:                           4                                         
Covariance Type:                  HC3                                         
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

# Growth outcomes with demographic controls
Here is an analysis that adds some demographic controls to see if this has an effect on their the intervention.

In [52]:
model_growth_controls = smf.ols(
    "BM1_gain_score ~ C(intervention) + pretest_score + ly_math_AASA_score + C(FRL) + C(ELL) + C(RaceEthnicity) + C(SPED)",
    data=df
).fit(cov_type="HC3")

print(model_growth_controls.summary().as_text())


                            OLS Regression Results                            
Dep. Variable:         BM1_gain_score   R-squared:                       0.442
Model:                            OLS   Adj. R-squared:                  0.424
Method:                 Least Squares   F-statistic:                     34.57
Date:                Tue, 03 Feb 2026   Prob (F-statistic):           9.10e-58
Time:                        15:59:26   Log-Likelihood:                -1808.6
No. Observations:                 422   AIC:                             3645.
Df Residuals:                     408   BIC:                             3702.
Df Model:                          13                                         
Covariance Type:                  HC3                                         
                                                           coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------

BM1 Score outcomes

In [11]:
model_bm1 = smf.ols(
    "bm1_score ~ C(intervention) + pretest_score + ly_math_AASA_score",
    data=df
).fit(cov_type="HC3")

print(model_bm1.summary())

                            OLS Regression Results                            
Dep. Variable:              bm1_score   R-squared:                       0.258
Model:                            OLS   Adj. R-squared:                  0.251
Method:                 Least Squares   F-statistic:                     36.59
Date:                Tue, 03 Feb 2026   Prob (F-statistic):           3.12e-26
Time:                        13:54:58   Log-Likelihood:                -1819.3
No. Observations:                 423   AIC:                             3649.
Df Residuals:                     418   BIC:                             3669.
Df Model:                           4                                         
Covariance Type:                  HC3                                         
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

In [33]:
# 1) Are there missing values?
df[['ly_math_AASA_score', 'pretest_score', 'BM1_gain_score','intervention']].isna().sum()


ly_math_AASA_score     7
pretest_score         11
BM1_gain_score        17
intervention           0
dtype: int64

In [34]:
# missing by group
df.groupby('intervention')['BM1_gain_score'].apply(lambda x: x.isna().sum()).to_frame('n_missing')


,n_missing
intervention,
In-Person,2
Lab,14
Online,1


In [30]:
# compare baseline vars by missingness
missing = df['BM1_gain_score'].isna()
df.groupby(missing)[['pretest_score','ly_math_AASA_score']].agg(['count','mean','std'])


pretest_score                         ly_math_AASA_score  \
                       count         mean        std              count   
BM1_gain_score                                                            
False                    428  3662.897196  18.639529                422   
True                       6  3677.333333  28.973551                 16   

                                        
                       mean        std  
BM1_gain_score                          
False           3640.590047  19.513810  
True            3640.000000  13.165612

In [21]:
# test whether missingness is predictable (logit)
df['missing'] = df['BM1_gain_score'].isna().astype(int)
m = smf.logit('missing ~ pretest_score + ly_math_AASA_score + C(intervention)', data=df).fit(disp=False)
print(m.summary())

                           Logit Regression Results                           
Dep. Variable:                missing   No. Observations:                  427
Model:                          Logit   Df Residuals:                      422
Method:                           MLE   Df Model:                            4
Date:                Tue, 03 Feb 2026   Pseudo R-squ.:                 0.03900
Time:                        14:06:02   Log-Likelihood:                -26.146
converged:                      False   LL-Null:                       -27.207
Covariance Type:            nonrobust   LLR p-value:                    0.7133
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                      -90.5228    109.303     -0.828      0.408    -304.752     123.706
C(intervention)[T.In-Person]     0.0919      1.128      0.081      0.935     

c:\Users\rcarlisle\OneDrive - qcusd.org\Reports\Summer Bridge\2026 Summer Bridge\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [13]:
# 2) Group counts and spread
df.groupby('intervention')['BM1_gain_score'].agg(['count','mean','std','var'])

,count,mean,std,var
intervention,,,,
Lab,308,-3.866883,23.913494,571.855186
In-Person,70,-8.228571,21.517887,463.019462
Online,60,-5.900000,22.126524,489.583051


In [14]:
# 3) Simple one-way ANOVA residual info
import statsmodels.formula.api as smf
m = smf.ols("BM1_gain_score ~ C(intervention)", data=df.dropna(subset=['BM1_gain_score','intervention'])).fit()
print("mse_resid:", m.mse_resid, "df_resid:", m.df_resid)

mse_resid: 543.4328392297357 df_resid: 435.0


In [23]:
df["BM1_gain_score"]

0       NaN
1      -7.0
2       4.0
3       8.0
4      -6.0
       ... 
526   -26.0
527   -23.0
528   -12.0
529    -6.0
530   -39.0
Name: BM1_gain_score, Length: 531, dtype: float64

In [24]:
df["intervention"]

0      In-Person
1      In-Person
2      In-Person
3      In-Person
4      In-Person
         ...    
526          Lab
527          Lab
528          Lab
529          Lab
530          Lab
Name: intervention, Length: 531, dtype: category
Categories (3, str): ['Lab', 'In-Person', 'Online']

# Effect Sizes
Effect sizes for the adjusted comparisons between interventions were small (Cohen’s d < 0.2), consistent with the non-significant differences observed in the regression models. This suggests that all three interventions had similar impacts on BM1 scores and growth.

In [30]:
resid_sd = np.sqrt(model_growth.mse_resid)

effects = model_growth.params.filter(like="intervention")

effects / resid_sd


C(intervention)[T.Lab]       0.192185
C(intervention)[T.Online]    0.075550
dtype: float64

# pairwise Cohen’s d helper

In [31]:
def cohens_d(g1, g2):
    x1 = df[df.intervention==g1]["BM1_gain_score"]
    x2 = df[df.intervention==g2]["BM1_gain_score"]
    pooled = np.sqrt((x1.var()+x2.var())/2)
    return (x1.mean()-x2.mean())/pooled

pairs = [("Lab","In-Person"), ("Lab","Online"), ("In-Person","Online")]

for a,b in pairs:
    print(a, "vs", b, cohens_d(a,b))

Lab vs In-Person 0.1917457520545387
Lab vs Online 0.0882531022073947
In-Person vs Online -0.10669613824981819


# Subgroup analyses

In [ ]:
model_sub = smf.ols(
    "BM1_gain_score ~ C(intervention)*C(school_name) + ly_math_AASA_score",
    data=df
).fit(cov_type="HC3")

print(model_sub.summary())

                            OLS Regression Results                            
Dep. Variable:         BM1_gain_score   R-squared:                       0.328
Model:                            OLS   Adj. R-squared:                  0.310
Method:                 Least Squares   F-statistic:                     22.84
Date:                Tue, 03 Feb 2026   Prob (F-statistic):           1.47e-36
Time:                        13:26:04   Log-Likelihood:                -1871.6
No. Observations:                 428   AIC:                             3767.
Df Residuals:                     416   BIC:                             3816.
Df Model:                          11                                         
Covariance Type:                  HC3                                         
                                                                              coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

c:\Users\rcarlisle\OneDrive - qcusd.org\Reports\Summer Bridge\2026 Summer Bridge\.venv\Lib\site-packages\statsmodels\regression\linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
c:\Users\rcarlisle\OneDrive - qcusd.org\Reports\Summer Bridge\2026 Summer Bridge\.venv\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 15, but rank is 11
  warnings.warn('covariance of constraints does not have full '


# Separate Regressions by Subgroup

In [34]:
results = []

for school, sub in df.groupby("school_name"):
    m = smf.ols(
        "BM1_gain_score ~ C(intervention) + pretest_score",
        data=sub
    ).fit()
    
    sd = np.sqrt(m.mse_resid)
    
    for term in m.params.index:
        if "C(intervention)" in term:
            results.append({
                "school": school,
                "comparison": term,
                "effect": m.params[term],
                "effect_size": m.params[term]/sd,
                "p": m.pvalues[term]
            })

pd.DataFrame(results)

IndexError: tuple index out of range